# Maschinelle Intelligenz und Gesellschaft 2023
Name:
누가 어떤 역할 했는지....


# FYI !!!
마크다운에서 코드! 쓰려면 맥북 cmd + space bar 누르고 terminal 검색해서 원화를 백쿼트(```code```)로 바꿔야함!

독일 맥북은 the top-right corner of the keyboard while holding down shift


terminal 에 아래 코드 긁어서 적용 후 reboot하면 원화 누를때마다 백쿼트 누를수 있음! 

{
    "₩" = ("insertText:", "`");
}

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

## Preprocessing

In [2]:
df = pd.read_csv('NC_policing.csv')

print(df.dtypes)
print(df.isna().sum())  # check for missing values

state                  object
stop_date              object
driver_gender          object
driver_age            float64
driver_race_raw        object
driver_race            object
violation              object
search_type            object
contraband_found         bool
stop_outcome           object
search_basis           object
officer_id              int64
drugs_related_stop     object
district               object
dtype: object
state                      0
stop_date                  0
driver_gender              0
driver_age                91
driver_race_raw            0
driver_race                0
violation                  0
search_type                0
contraband_found           0
stop_outcome               0
search_basis               0
officer_id                 0
drugs_related_stop    397708
district                  38
dtype: int64


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402087 entries, 0 to 402086
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   state               402087 non-null  object 
 1   stop_date           402087 non-null  object 
 2   driver_gender       402087 non-null  object 
 3   driver_age          401996 non-null  float64
 4   driver_race_raw     402087 non-null  object 
 5   driver_race         402087 non-null  object 
 6   violation           402087 non-null  object 
 7   search_type         402087 non-null  object 
 8   contraband_found    402087 non-null  bool   
 9   stop_outcome        402087 non-null  object 
 10  search_basis        402087 non-null  object 
 11  officer_id          402087 non-null  int64  
 12  drugs_related_stop  4379 non-null    object 
 13  district            402049 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(11)
memory usage: 40.3+ MB


In [3]:
df.head()

,state,stop_date,driver_gender,driver_age,driver_race_raw,driver_race,violation,search_type,contraband_found,stop_outcome,search_basis,officer_id,drugs_related_stop,district
0,NC,2000-07-31,M,23.0,W N,White,Speeding,No Search Conducted,False,Arrest,No Search Conducted,10606,NaN,E3
1,NC,2000-07-31,F,29.0,W N,White,Speeding,No Search Conducted,False,Arrest,No Search Conducted,10606,NaN,E3
2,NC,2000-07-31,F,49.0,B N,Black,Speeding,No Search Conducted,False,Arrest,No Search Conducted,10606,NaN,E3
3,NC,2000-07-31,M,29.0,B N,Black,Speeding,No Search Conducted,False,Arrest,No Search Conducted,10606,NaN,E3
4,NC,2000-07-31,M,24.0,U H,Hispanic,Speeding,No Search Conducted,False,Arrest,No Search Conducted,10606,NaN,E3


In [5]:
#Distribution of Driver Race in the Data
plt.figure(figsize=(10,5))
plt.hist(data['violation'], label='Driver Race')
plt.legend()
plt.ylabel('Frequency')
plt.xlabel('Driver Race')
plt.title("Distribution of Driver Race in the Data")
plt.show()

KeyError: 'M'

### Based on the basic data information, we can deduce:

- Total Entries: 403087 entries in total.
- Missing Values: 91 in **'driver_age'**, 397708 in **'drugs_related_stop'** and 38 in **'district'**.
- Data Types: Most columns have the object as string, **'driver_age'** and **'officer_id'** as numeric(float and int), ans one boolean type **'contraband_found'**.

### Features that we could drop:

- **state** : The dataset is based on North carolina (NC) also we dont need this column. drop it.
- either **'4 driver_race_raw'** or **'5 driver_race'** : These two are presumed to be similar information, Dh. one of them drop
- **'rugs_related_stop'**: There are so many missing values. it will be no meaningful dataset.
- **'driver_race_raw'** combines two codes to give a more specific race and origin information and **'driver_race'** priveds more simpler and vivid classification of race, with five clear categories: White, Black, Hispanic, Other and Asian. Because of siplification and interpretability we will use **'driver_race'** column and remove **'driver_race_raw'**.

### Conclusion:
We will drop **'state'** and **'drugs_related_stop'** and **'driver_race_raw'**.

In [19]:
# drop unnecessary values
columns_to_drop = ['state', 'drugs_related_stop', 'driver_race_raw']
df.drop(columns=columns_to_drop, inplace=True)


print(df.head())

    stop_date driver_gender  driver_age driver_race violation  \
0  2000-07-31             M        23.0       White  Speeding   
1  2000-07-31             F        29.0       White  Speeding   
2  2000-07-31             F        49.0       Black  Speeding   
3  2000-07-31             M        29.0       Black  Speeding   
4  2000-07-31             M        24.0    Hispanic  Speeding   

           search_type  contraband_found stop_outcome         search_basis  \
0  No Search Conducted             False       Arrest  No Search Conducted   
1  No Search Conducted             False       Arrest  No Search Conducted   
2  No Search Conducted             False       Arrest  No Search Conducted   
3  No Search Conducted             False       Arrest  No Search Conducted   
4  No Search Conducted             False       Arrest  No Search Conducted   

   officer_id district  
0       10606       E3  
1       10606       E3  
2       10606       E3  
3       10606       E3  
4       10606  

In [20]:
# stop_date to date format
df['stop_date'] = pd.to_datetime(df['stop_date'])
df['year'] = df['stop_date'].dt.year
df['month'] = df['stop_date'].dt.month
df['day'] = df['stop_date'].dt.day
df.drop(columns='stop_date', inplace=True)

# stop_outcome to binary format
outcome_mapping = {"Arrest": 1, "Not Arrested": 0} 
df['stop_outcome'] = df['stop_outcome'].replace(outcome_mapping)

# NaN in driver_age to median
df['driver_age'].fillna(df['driver_age'].median(), inplace=True)

categorical_columns = ['driver_gender', 'driver_race', 'violation', 'search_type', 'search_basis', 'district']
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)


In [ ]:
# for Missing values
df.dropna(inplace=True)

#Converting
# 'stop_date' to datetime formation.
# extract 년/월/일 as separate features
# 각각 따로 분류하면 년/월/일에 경향성 학습 가능성 높아져
df['stop_date'] = pd.to_datetime(df['stop_date'])
df['year'] = df['stop_date'].dt.year
df['month'] = df['stop_date'].dt.month
df['day'] = df['stop_date'].dt.day
df.drop(columns='stop_date', inplace=True)

# target column 'stop_outcome' to binary format (Arrest=1, Other=0)
stop_outcome_replacements = {"Arrest": 1, "Not Arrest": 0}
df['stop_outcome'] = df['stop_outcome'].replace(stop_outcome_replacements)

# for Missing values
#df.dropna(inplace=True)

# Use LabelEncoder for binary categorical columns
le = LabelEncoder()
df['driver_gender'] = le.fit_transform(df['driver_gender'])
df['contraband_found'] = le.fit_transform(df['contraband_found'])

# One-hot encoding for other categorical columns
df = pd.get_dummies(df, columns=['driver_race', 'violation', 'search_type', 'search_basis', 'district'])



-labelencoder : binary category사용, M = 1 / F = 0으로 변환 가능 numeric variable로 변환되면 모델에 적용 가능
-one-hot encoding : multiple category에 사용(=2개 이상의 값)```pd.get_dummies``` **(Assignment02!)** 사용
-stop_date convert : 연/월/일 분해시 경향성 학습 굿

이거는 리포트에 써야할듯..

In [21]:
df.head()

,driver_age,contraband_found,stop_outcome,officer_id,year,month,day,driver_gender_M,driver_race_Black,driver_race_Hispanic,...,district_G4,district_G5,district_G6,district_G7,district_H1,district_H2,district_H3,district_H4,district_H5,district_H6
0,23.0,False,1,10606,2000,7,31,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,29.0,False,1,10606,2000,7,31,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,49.0,False,1,10606,2000,7,31,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,29.0,False,1,10606,2000,7,31,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,24.0,False,1,10606,2000,7,31,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df.isna().sum()

driver_age          0
contraband_found    0
stop_outcome        0
officer_id          0
year                0
                   ..
district_H2         0
district_H3         0
district_H4         0
district_H5         0
district_H6         0
Length: 112, dtype: int64

NaN을 fillna / dropna / interpolate 뭘 써야할지 모르겟댜..

In [23]:
df.dtypes

driver_age          float64
contraband_found       bool
stop_outcome         object
officer_id            int64
year                  int64
                     ...   
district_H2           uint8
district_H3           uint8
district_H4           uint8
district_H5           uint8
district_H6           uint8
Length: 112, dtype: object

In [27]:
df.head()

,driver_age,contraband_found,stop_outcome,officer_id,year,month,day,driver_gender_M,driver_race_Black,driver_race_Hispanic,...,district_G4,district_G5,district_G6,district_G7,district_H1,district_H2,district_H3,district_H4,district_H5,district_H6
0,23.0,False,1,10606,2000,7,31,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,29.0,False,1,10606,2000,7,31,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,49.0,False,1,10606,2000,7,31,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,29.0,False,1,10606,2000,7,31,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,24.0,False,1,10606,2000,7,31,1,0,1,...,0,0,0,0,0,0,0,0,0,0


다시 data를 확인하면 모두 numeric으로 바뀐것을 알 수 있다. 

### Generate two binary classifiers using two different methods of your liking and compare their performances against a dummy classifier, which always predicts “not arrested”.

In [24]:
# data split to training set & test sets
X = df.drop('stop_outcome', axis=1)

y = (df['stop_outcome'] == "Arrested").astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [25]:
# Standardizing features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [26]:
# SVM Classifier
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_test)

ValueError: The number of classes has to be greater than one; got 1 class

In [ ]:
# Logistic Regression
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
log_reg_pred = log_reg.predict(X_test)

In [ ]:

# Dummy Classifier
dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(X_train, y_train)
dummy_pred = dummy.predict(X_test)



In [ ]:

# 성능 평가
print("SVM Classifier Performance:")
print(classification_report(y_test, svm_pred))
print("\nLogistic Regression Performance:")
print(classification_report(y_test, log_reg_pred))
print("\nDummy Classifier Performance:")
print(classification_report(y_test, dummy_pred))
